In [134]:
import os, requests
from dotenv import load_dotenv
# Load environment variables from the .env file
load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

search_en_id = os.getenv('search_en_id')
api_key = os.getenv('api_key')

In [260]:
import google.generativeai as genai
from langchain.tools import DuckDuckGoSearchResults
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

from langchain_google_genai import (ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings,HarmBlockThreshold,HarmCategory)

from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores import faiss

from langchain_community.document_loaders import WebBaseLoader

In [149]:
safe ={
	HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
	HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
	HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
	HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
	}

generation_config = {
  "temperature": 0.9,
  "top_p": 1,
  "top_k": 1,
  "max_output_tokens": 10000,
}

In [3]:
for m in genai.list_models():
	if 'generateContent' in m.supported_generation_methods:
		print(m.name)

model = "gemini-1.0-pro-vision-latest"

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-pro
models/gemini-pro-vision


In [133]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [139]:
url_ = "https://www.googleapis.com/customsearch/v1"

def build_params(search_query, num = 10, start=1, dateRestrict='d1',**kwargs):
	params = { 'q': search_query, 
		   	   'key': api_key, 
			   'cx': search_en_id,
			   'num': num,
			   'start': 1,
			   'dateRestrict': 'd1',
			   }
	params.update(kwargs)
	return params

def google_search(params):
	response = requests.get(url_, params=params)
	if response.status_code != 200:
		raise Exception('API response: {}'.format(response.status_code))
	return response.json()

In [6]:
# search = DuckDuckGoSearchResults()
# res = search.run("Obamna", max_results=2)
# wrapper = DuckDuckGoSearchAPIWrapper( max_results=2)
# search = DuckDuckGoSearchResults(api_wrapper=wrapper, source="news")
# res = search.run("Obamna")

# Searching for pdf files
# results = DDGS().text('russia filetype:pdf', region='us-en', safesearch='off', timelimit='y', max_results=10)

# async


In [131]:
query = "obamna"

In [108]:
from duckduckgo_search import DDGS

results = DDGS().text('Obamna', region='us-en', safesearch='off', timelimit='y', max_results=20)


In [132]:
results[:2]

[{'title': 'Barack Obama: Biography, 44th U.S. President, Politician',
  'href': 'https://www.biography.com/political-figures/barack-obama',
  'body': 'Learn about the life and achievements of Barack Obama, the first Black American to be elected president of the United States. Find out about his early years, education, family, political career, Nobel Peace Prize, books, hobbies and more.'},
 {'title': 'Barack Obama | Biography, Parents, Education, Presidency, Books ...',
  'href': 'https://www.britannica.com/biography/Barack-Obama',
  'body': 'Barack Obama (born August 4, 1961, Honolulu, Hawaii, U.S.) 44th president of the United States (2009-17) and the first African American to hold the office. Before winning the presidency, Obama represented Illinois in the U.S. Senate (2005-08). He was the third African American to be elected to that body since the end of Reconstruction (1877).'}]

In [110]:
meta = [{"href": f"{r['href']}", "title": f"{r['title']}"} for r in results]
txt = [r['body'] for r in results]

In [135]:
meta[:2]

[{'href': 'https://www.biography.com/political-figures/barack-obama',
  'title': 'Barack Obama: Biography, 44th U.S. President, Politician'},
 {'href': 'https://www.britannica.com/biography/Barack-Obama',
  'title': 'Barack Obama | Biography, Parents, Education, Presidency, Books ...'}]

In [112]:
db = faiss.FAISS.from_texts(txt, embeddings, metadatas=meta)

In [114]:
docs_and_scores = db.similarity_search_with_score(query, k=10)

In [137]:
docs_and_scores[::-1][:2]

[(Document(page_content='Learn about the life and achievements of Barack Obama, the first Black American to be elected president of the United States. Find out about his early years, education, family, political career, Nobel Peace Prize, books, hobbies and more.', metadata={'href': 'https://www.biography.com/political-figures/barack-obama', 'title': 'Barack Obama: Biography, 44th U.S. President, Politician'}),
  0.5331205),
 (Document(page_content="Former president Barack Obama promises in a White House lunch to do all he can for President Biden's reelection. By Tyler Pager. August 2, 2023 at 12:18 p.m. EDT. Former president Barack Obama ...", metadata={'href': 'https://www.washingtonpost.com/politics/2023/08/02/obama-biden-campaign-reelection/', 'title': 'Obama promises Biden he will do all he can to get him reelected - The ...'}),
  0.5183651)]

In [130]:
links = [d[0].metadata['href'] for d in docs_and_scores[::-1]]
links

['https://www.biography.com/political-figures/barack-obama',
 'https://www.washingtonpost.com/politics/2023/08/02/obama-biden-campaign-reelection/',
 'https://www.nytimes.com/2023/05/31/us/politics/obama-oral-history.html',
 'https://www.latimes.com/opinion/story/2024-03-18/great-expectations-vinson-cunningham-obama-campaign',
 'https://www.britannica.com/biography/Michelle-Obama',
 'https://knowyourmeme.com/editorials/guides/where-did-the-nickname-obamna-come-from-memes-about-trumps-mispronounciation-of-obamas-name-explained',
 'https://www.cnn.com/2024/01/20/politics/obama-inauguration-america-blake-cec/index.html',
 'https://www.youtube.com/watch?v=NwLOuwtPAJ4',
 'https://www.forbes.com/advisor/health-insurance/what-is-obamacare/',
 'https://www.britannica.com/biography/Barack-Obama']

### Google

In [140]:
response = google_search(build_params(query,num = 10))

In [142]:
for i in response['items']:
	url, title = i['link'], i['title']
	print(f"URL : {url}  title : {title}")
	print(f"Snippet : {i['snippet']}")

URL : https://twitter.com/GarakObamna/status/1769384400918335896  title : Garak Obamna on X: "a gentle polemic against @OGRolandRat's ...
Snippet : 23 hours ago ... Garak Obamna · @GarakObamna. a gentle polemic against. @OGRolandRat. 's reading of MAGA. MAGA means Nationalism, not the Kali Yuga · From substackfwd.xyz.
URL : https://www.reddit.com/r/thomastheplankengine/comments/1bhbjy0/i_had_a_dream_that_this_screenshot_went_viral/  title : I had a dream that this screenshot went viral because nobody could ...
Snippet : 16 hours ago ... ... it clearly says Obama. Upvote 1. Downvote Reply reply. Share. u/Terracatlegend avatar · Terracatlegend. • in 23h. Obamna. Upvote 7. Downvote Reply reply. Share.
URL : https://twitter.com/GarakObamna/status/1769526225440264597  title : Garak Obamna on X: "Please God make this happen" / X
Snippet : 14 hours ago ... Spanian is becoming an icon. He has done some great videos on some of the most extreme communities in the world. I think its time he does 

In [144]:
loader = WebBaseLoader(links)
docs = loader.load()
docs

[Document(page_content='Barack Obama: Biography, 44th U.S. President, PoliticianSearchWomen’s HistoryHistory & CultureMusiciansMovies & TVAthletesArtistsPower & PoliticsBusinessScholars & EducatorsScientistsActivistsNotorious FiguresBIO BuysNewsletterYour Privacy ChoicesPrivacy NoticeTerms Of UseSkip to ContentWomen’s HistoryMusiciansMovies & TVAthletesNewsletterFamous Political FiguresU.S. PresidentsBarack ObamaBarack ObamaThe 44th president of the United States, Barack Obama is the first Black American who has been elected to the Oval Office. He served from 2009 until 2017.By Tyler Piccotti and Biography.com EditorsUpdated: May 1, 2023Courtesy The White HouseWe may earn commission from links on this page, but we only recommend products we back.Jump to:Who Is Barack Obama?Quick FactsEarly Life and FamilyEducationMarriage to Michelle Obama and DaughtersIllinois Political Career2008 Presidential Election and Inauguration First Term as U.S. PresidentSecond Term as U.S. PresidentNotable S

In [182]:
from bs4 import BeautifulSoup
from readabilipy import simple_json_from_html_string
from urllib.request import urlopen

In [145]:
def html_txt(url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:77.0) Gecko/20190101 Firefox/77.0'}
        html = requests.get(url, headers=headers, timeout=2).content
    except Exception as e:
            print(f"Error : {e}")
    return BeautifulSoup(html, "html.parser").getText().strip()


In [155]:
def html_txt(url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
        response = requests.get(url, headers=headers, timeout=5)
        response.raise_for_status()  # Raise an exception if the status code is not 200
        html = response.content
    except requests.exceptions.RequestException as e:
        print(f"Error: {e} for url : {url}")
        return None
    soup = BeautifulSoup(html, "html.parser")
    return soup.get_text().strip()


In [173]:
def html_txt(url):
	try:
		headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
		response = requests.get(url, headers=headers, timeout=5)
		response.raise_for_status()  # Raise an exception if the status code is not 200
		html = response.text
	except requests.exceptions.RequestException as e:
		print(f"Error: {e} for url : {url}")
		return None
	return get_plain_text(html)

def get_plain_text(html):
	article = simple_json_from_html_string(html, use_readability=True)
	# article['plain_text']
	text = "\n".join([tx['text'].strip("\n") for tx in article['plain_text']])
	return text

In [212]:
from langchain_community.document_loaders import SeleniumURLLoader
# from langchain_community.document_loaders import PlaywrightURLLoader

loader = SeleniumURLLoader(urls=links[:5])

# loader = PlaywrightURLLoader(urls=links, remove_selectors=["header", "footer"])






In [213]:
%%time
data = loader.load()

CPU times: user 1.96 s, sys: 107 ms, total: 2.06 s
Wall time: 1min 40s


In [214]:
data

[Document(page_content='Famous Political Figures\n\nU.S. Presidents\n\nBarack Obama\n\nBarack Obama\n\nThe 44\n\nth president of the United States, Barack Obama is the first Black American who has been elected to the Oval Office. He served from 2009 until 2017.\n\nBy Tyler Piccotti and Biography.com Editors\n\nCourtesy The White House\n\nWe may earn commission from links on this page, but we only recommend products we back.\n\nJump to:\n\nWho Is Barack Obama?\n\nQuick Facts\n\nEarly Life and Family\n\nEducation\n\nMarriage to Michelle Obama and Daughters\n\nIllinois Political Career\n\n2008 Presidential Election and Inauguration\n\nFirst Term as U.S. President\n\nSecond Term as U.S. President\n\nNotable Speeches\n\nLife After the Presidency\n\nHow Tall is Obama?\n\nBooks and Grammy\n\nHobbies\n\nMovies About Obama\n\nQuotes\n\nWho Is Barack Obama?\n\nawarded the Nobel Peace Prize. He and his wife,\n\nMichelle Obama, have two daughters,\n\nMalia and\n\nSasha.\n\nQuick Facts\n\nFULL NAME

In [225]:
%%time
loader = WebBaseLoader(links[8])
d = loader.load()
d

CPU times: user 34.4 ms, sys: 12.1 ms, total: 46.5 ms
Wall time: 118 ms


[Document(page_content='Error PageSecurity Violation (403)\n\nfbadvisor-release-reblaze.forbes.com\xa0|\xa0Access denied (403)\nCurrent session has been terminated.\nFor further information, do not hesitate to contact us.\nRef: 108.31.204.45\xa01710786626\n\n\n', metadata={'source': 'https://www.forbes.com/advisor/health-insurance/what-is-obamacare/', 'title': 'Error Page', 'language': 'No language found.'})]

In [261]:
%%time
loader = WebBaseLoader(links[8])
d = loader.load()
d

CPU times: user 1.6 s, sys: 0 ns, total: 1.6 s
Wall time: 1.83 s


[Document(page_content="\n\n\n\n\n\n\n\n\n\n\n\nWhat Is The Affordable Care Act (Obamacare)? – Forbes Advisor\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n  \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\t\t\t\t\tSelect Region\n\t\t\t\t\n\n\n\n\n\n\n\n\n\n\nUnited States\n\n\n\n\n\n\n\n\n\n\nUnited Kingdom\n\n\n\n\n\n\n\n\n\n\nGermany\n\n\n\n\n\n\n\n\n\n\nIndia\n\n\n\n\n\n\n\n\n\n\nAustralia\n\n\n\n\n\n\n\n\n\n\nItaly\n\n\n\n\n\n\n\n\n\n\nCanada\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                    Credit Cards                \n\n\n\n\n\n\n\n\nCredit Cards\n\n\n\n\n                                    Credit Cards                                \n\n\n\n\n                                            Compare Credit Cards                                            \n\n\n\n                                                    Best Credit Cards                                                \

In [232]:
d[0].page_content

"\n\n\n\n\n\n\n\n\n\n\n\nWhat Is The Affordable Care Act (Obamacare)? – Forbes Advisor\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n  \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\t\t\t\t\tSelect Region\n\t\t\t\t\n\n\n\n\n\n\n\n\n\n\nUnited States\n\n\n\n\n\n\n\n\n\n\nUnited Kingdom\n\n\n\n\n\n\n\n\n\n\nGermany\n\n\n\n\n\n\n\n\n\n\nIndia\n\n\n\n\n\n\n\n\n\n\nAustralia\n\n\n\n\n\n\n\n\n\n\nItaly\n\n\n\n\n\n\n\n\n\n\nCanada\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                    Credit Cards                \n\n\n\n\n\n\n\n\nCredit Cards\n\n\n\n\n                                    Credit Cards                                \n\n\n\n\n                                            Compare Credit Cards                                            \n\n\n\n                                                    Best Credit Cards                                                \n\n                    

In [217]:
import nest_asyncio

nest_asyncio.apply()

In [247]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:123.0) Gecko/20100101 Firefox/123.0'}

requests_kwargs = headers

In [262]:
loader = WebLoader(links, requests_kwargs=requests_kwargs)
loader.requests_kwargs.update(headers)
loader.requests_per_second = 5


In [264]:
links

['https://www.biography.com/political-figures/barack-obama',
 'https://www.washingtonpost.com/politics/2023/08/02/obama-biden-campaign-reelection/',
 'https://www.nytimes.com/2023/05/31/us/politics/obama-oral-history.html',
 'https://www.latimes.com/opinion/story/2024-03-18/great-expectations-vinson-cunningham-obama-campaign',
 'https://www.britannica.com/biography/Michelle-Obama',
 'https://knowyourmeme.com/editorials/guides/where-did-the-nickname-obamna-come-from-memes-about-trumps-mispronounciation-of-obamas-name-explained',
 'https://www.cnn.com/2024/01/20/politics/obama-inauguration-america-blake-cec/index.html',
 'https://www.youtube.com/watch?v=NwLOuwtPAJ4',
 'https://www.forbes.com/advisor/health-insurance/what-is-obamacare/',
 'https://www.britannica.com/biography/Barack-Obama']

In [263]:
%%time
docs = loader.aload()
docs

Fetching pages:   0%|          | 0/10 [00:00<?, ?it/s]

Fetching pages: 100%|##########| 10/10 [00:00<00:00, 13.26it/s]


CPU times: user 712 ms, sys: 33 ms, total: 745 ms
Wall time: 1.16 s


[Document(page_content='Barack Obama: Biography, 44th U.S. President, PoliticianSearchWomen’s HistoryHistory & CultureMusiciansMovies & TVAthletesArtistsPower & PoliticsBusinessScholars & EducatorsScientistsActivistsNotorious FiguresBIO BuysNewsletterYour Privacy ChoicesPrivacy NoticeTerms Of UseSkip to ContentWomen’s HistoryMusiciansMovies & TVAthletesNewsletterFamous Political FiguresU.S. PresidentsBarack ObamaBarack ObamaThe 44th president of the United States, Barack Obama is the first Black American who has been elected to the Oval Office. He served from 2009 until 2017.By Tyler Piccotti and Biography.com EditorsUpdated: May 1, 2023Courtesy The White HouseWe may earn commission from links on this page, but we only recommend products we back.Jump to:Who Is Barack Obama?Quick FactsEarly Life and FamilyEducationMarriage to Michelle Obama and DaughtersIllinois Political Career2008 Presidential Election and Inauguration First Term as U.S. PresidentSecond Term as U.S. PresidentNotable S

In [191]:
loader = WebBaseLoader(links[0])


data = loader.load()

data

[Document(page_content='Barack Obama: Biography, 44th U.S. President, PoliticianSearchWomen’s HistoryHistory & CultureMusiciansMovies & TVAthletesArtistsPower & PoliticsBusinessScholars & EducatorsScientistsActivistsNotorious FiguresBIO BuysNewsletterYour Privacy ChoicesPrivacy NoticeTerms Of UseSkip to ContentWomen’s HistoryMusiciansMovies & TVAthletesNewsletterFamous Political FiguresU.S. PresidentsBarack ObamaBarack ObamaThe 44th president of the United States, Barack Obama is the first Black American who has been elected to the Oval Office. He served from 2009 until 2017.By Tyler Piccotti and Biography.com EditorsUpdated: May 1, 2023Courtesy The White HouseWe may earn commission from links on this page, but we only recommend products we back.Jump to:Who Is Barack Obama?Quick FactsEarly Life and FamilyEducationMarriage to Michelle Obama and DaughtersIllinois Political Career2008 Presidential Election and Inauguration First Term as U.S. PresidentSecond Term as U.S. PresidentNotable S

In [186]:
txt

['We may earn commission from links on this page, but we only recommend products we back.\n1961-present\nWho Is Barack Obama?\nBarack Obama was the 44th president of the United States and the first Black commander-in-chief. He served two terms, from 2009 until 2017. The son of parents from Kenya and Kansas, Obama was born and raised in Hawaii. He graduated from Columbia University and Harvard Law School, where he was president of the Harvard Law Review. After serving on the Illinois State Senate, he was elected a U.S. senator representing Illinois in 2004. In 2009, Obama was awarded the Nobel Peace Prize. He and his wife, Michelle Obama, have two daughters, Malia and Sasha.\nQuick Facts\nFULL NAME: Barack Hussein Obama IIBORN: August 4, 1961BIRTHPLACE: Honolulu, HawaiiSPOUSE: Michelle Obama (1992-present)CHILDREN: Malia and SashaASTROLOGICAL SIGN: Leo\nEarly Life and Family\nBarack Hussein Obama II was born on August 4, 1961, in Honolulu to Barack Obama Sr. and Ann Dunham. He has six h

In [178]:
txt[-1]

'CNN —\nElizabeth Alexander was in her hotel room in Washington, DC, one frigid winter morning when she was awakened by a strange noise outside her window. She peered outside and saw a sea of people, bundled against the cold, walking in the predawn darkness towards the National Mall.\nIt was January 20, 2009, and the crowds were on their way to witness the inauguration of Barack Hussein Obama, the nation’s first Black president. The sound she heard was their footsteps, marching almost in unison as their numbers grew, which sounded to Alexander like the “growing rumble of thunder or a crashing wave.”\nAlexander had a coveted hotel room near the Mall that day because she was a special guest of Obama’s. He had asked Alexander, an author and poet who was then a professor at Yale University, to compose and recite a poem for his inaugural. Upon reaching the inaugural platform, Alexander saw she was sharing the stage with dignitaries such as boxing legend Muhammad Ali, singer Aretha Franklin,

In [147]:
results = DDGS().suggestions(query)

# async
results 

[{'phrase': 'obamna soda'},
 {'phrase': 'obamna meme'},
 {'phrase': 'obama mp3'},
 {'phrase': 'obama trump'},
 {'phrase': 'obama gaming'},
 {'phrase': 'obama sound'},
 {'phrase': 'obamacare'},
 {'phrase': 'obama amnesty'}]

### Suggestions/ related

In [152]:
model = genai.GenerativeModel('gemini-1.0-pro-latest')
response = model.generate_content("Generate a more detailed internet search query based on this key word: zaphod beeblebrox", safety_settings=safe)

print(response.text)

Zaphod Beeblebrox character analysis
Zaphod Beeblebrox quotes
Zaphod Beeblebrox personality type
Zaphod Beeblebrox leadership style
Zaphod Beeblebrox role in The Hitchhiker's Guide to the Galaxy
Zaphod Beeblebrox behind the scenes
Zaphod Beeblebrox actor
Zaphod Beeblebrox voice actor
Zaphod Beeblebrox fan fiction
Zaphod Beeblebrox fan art


In [164]:
def get_plain_text(html):
	article = simple_json_from_html_string(html, use_readability=True)
	# article['plain_text']
	text = "\n".join([tx['text'].strip("\n") for tx in article['plain_text']])
	return text

In [166]:
links

['https://www.biography.com/political-figures/barack-obama',
 'https://www.washingtonpost.com/politics/2023/08/02/obama-biden-campaign-reelection/',
 'https://www.nytimes.com/2023/05/31/us/politics/obama-oral-history.html',
 'https://www.latimes.com/opinion/story/2024-03-18/great-expectations-vinson-cunningham-obama-campaign',
 'https://www.britannica.com/biography/Michelle-Obama',
 'https://knowyourmeme.com/editorials/guides/where-did-the-nickname-obamna-come-from-memes-about-trumps-mispronounciation-of-obamas-name-explained',
 'https://www.cnn.com/2024/01/20/politics/obama-inauguration-america-blake-cec/index.html',
 'https://www.youtube.com/watch?v=NwLOuwtPAJ4',
 'https://www.forbes.com/advisor/health-insurance/what-is-obamacare/',
 'https://www.britannica.com/biography/Barack-Obama']

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:123.0) Gecko/20100101 Firefox/123.0'}

requests_kwargs = headers

In [252]:
from typing import Any, Union, Optional
from bs4 import BeautifulSoup
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:123.0) Gecko/20100101 Firefox/123.0'}

class WebLoader(WebBaseLoader):
    # self.requests_kwargs.update({"headers": headers})
    def _scrape(self, url: str, parser: Union[str, None] = None, bs_kwargs: Optional[dict] = None,) -> Any:
        
        
        if parser is None:
            if url.endswith(".xml"):
                parser = "xml"
            else:
                parser = self.default_parser
        
        self._check_parser(parser)
        
        html_doc = self.session.get(url, **self.requests_kwargs)
        if self.raise_for_status:
            html_doc.raise_for_status()
        
        html_doc.encoding = html_doc.apparent_encoding
        
        return BeautifulSoup(html_doc.text, parser)